# Loading Libraries

In [25]:
import torch, pandas, scipy 
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn as nn
from csv import reader

# Data Preprocessing 

In [26]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [27]:
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [28]:
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
            row[column] = lookup[row[column]]
    return lookup

In [29]:
# load and prepare data
#xy = np.loadtxt('QCM Sensor Alcohol Dataset/QCM10.csv', delimiter=";", dtype=np.float32, skiprows=1)
#X = xy[:,0:14]
#y = xy[:, [0]]
X = torch.tensor(X, dtype = torch.float)
y = torch.tensor(y, dtype = torch.float)
xPredicted = torch.tensor(X, dtype = torch.float)

print(X.size())
print(y.size())

torch.Size([25, 14])
torch.Size([25, 1])


<ipython-input-29-47ebf1f30d55>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xPredicted = torch.tensor(X, dtype = torch.float)


In [30]:
X_max, _ = torch.max(X, 0)
xPredicted_max, _ = torch.max(xPredicted, 0)

X = torch.div(X, X_max)
xPredicted_max = torch.div(xPredicted, xPredicted_max)
y = y / 100

# BPNN Model

In [31]:
class Neural_Network(nn.Module):
    def __init__(self, ):
        super(Neural_Network, self).__init__()
        # parameters
        # TODO: parameters can be parameterized instead of declaring them here
        self.inputSize = 14
        self.outputSize = 1
        self.hiddenSize = 25
        
        # weights
        self.W1 = torch.randn(self.inputSize, self.hiddenSize) # 2 X 3 tensor
        self.W2 = torch.randn(self.hiddenSize, self.outputSize) # 3 X 1 tensor
        
    def forward(self, X):
        self.z = torch.matmul(X, self.W1) # 3 X 3 ".dot" does not broadcast in PyTorch
        self.z2 = self.sigmoid(self.z) # activation function
        self.z3 = torch.matmul(self.z2, self.W2)
        o = self.sigmoid(self.z3) # final activation function
        return o
        
    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))
    
    def sigmoidPrime(self, s):
        # derivative of sigmoid
        return s * (1 - s)
    
    def backward(self, X, y, o):
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # derivative of sig to error
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.z2)
        self.W1 += torch.matmul(torch.t(X), self.z2_delta)
        self.W2 += torch.matmul(torch.t(self.z2), self.o_delta)
        
    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)
        
    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")
        # you can reload model with all the weights and so forth with:
        # torch.load("NN")
        
    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(xPredicted)))

In [32]:
NN = Neural_Network()
for i in range(500):  # trains the NN 500 times
    print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X))**2).detach().item()))  # mean sum squared loss
    NN.train(X, y)
NN.saveWeights(NN)
NN.predict()

#0 Loss: 1.259617805480957
#1 Loss: 0.24852041900157928
#2 Loss: 0.24852041900157928
#3 Loss: 0.24852041900157928
#4 Loss: 0.24852041900157928
#5 Loss: 0.24852041900157928
#6 Loss: 0.24852041900157928
#7 Loss: 0.24852041900157928
#8 Loss: 0.24852041900157928
#9 Loss: 0.24852041900157928
#10 Loss: 0.24852041900157928
#11 Loss: 0.24852041900157928
#12 Loss: 0.24852041900157928
#13 Loss: 0.24852041900157928
#14 Loss: 0.24852041900157928
#15 Loss: 0.24852041900157928
#16 Loss: 0.24852041900157928
#17 Loss: 0.24852041900157928
#18 Loss: 0.24852041900157928
#19 Loss: 0.24852041900157928
#20 Loss: 0.24852041900157928
#21 Loss: 0.24852041900157928
#22 Loss: 0.24852041900157928
#23 Loss: 0.24852041900157928
#24 Loss: 0.24852041900157928
#25 Loss: 0.24852041900157928
#26 Loss: 0.24852041900157928
#27 Loss: 0.24852041900157928
#28 Loss: 0.24852041900157928
#29 Loss: 0.24852041900157928
#30 Loss: 0.24852041900157928
#31 Loss: 0.24852041900157928
#32 Loss: 0.24852041900157928
#33 Loss: 0.2485204190

C:\Users\User\anaconda3\envs\andirian98\lib\site-packages\torch\serialization.py:401: UserWarning: Couldn't retrieve source code for container of type Neural_Network. It won't be checked for correctness upon loading.
  warnings.warn("Couldn't retrieve source code for container of "
